# 第13章: プロパティとメソッドの詳細

## 学習目標
- アクセス修飾子を理解する
- ゲッターとセッターの使い方をマスターする
- 静的プロパティ・静的メソッドを理解する
- マジックメソッドの基礎を学ぶ

---

## 🎓 カプセル化の哲学

カプセル化とは、**「データを保護し、外部から直接操作できないようにすること」**です。

### なぜカプセル化が必要か？

```php
// 悪い例：誰でも年齢を変えられる
$user->age = -5;  // 負の年齢？不正な状態！
$user->email = "不正なアドレス";  // バリデーションなし

// 良い例：セッターで保護
$user->setAge(-5);  // エラーになる
$user->setEmail("invalid");  // バリデーションされる
```

**カプセル化のメリット**
1. 不正な値の代入を防げる
2. 内部実装を変更しても外側に影響しない
3. データの整合性を保てる
4. 保守性が向上する

---
## 1. アクセス修飾子

PHPには3つのアクセス修飾子があります。

| 修飾子 | クラス内 | サブクラス | 外部 |
|--------|---------|-----------|------|
| `public` | ○ | ○ | ○ |
| `protected` | ○ | ○ | × |
| `private` | ○ | × | × |

In [ ]:
<?php

class BankAccount {
    // public: どこからでもアクセス可能
    public string $ownerName;
    
    // protected: クラス内とサブクラスからのみアクセス可能
    protected string $accountNumber;
    
    // private: クラス内からのみアクセス可能
    private int $balance;
    private string $pinCode;
    
    public function __construct(string $owner, string $accountNumber, string $pin, int $initialBalance = 0) {
        $this->ownerName = $owner;
        $this->accountNumber = $accountNumber;
        $this->pinCode = $pin;
        $this->balance = $initialBalance;
    }
    
    // 残高を取得（privateなので外部から直接アクセスできない）
    public function getBalance(): int {
        return $this->balance;
    }
    
    // PIN認証付きで残高を取得
    public function getBalanceWithPin(string $pin): ?int {
        if ($this->verifyPin($pin)) {
            return $this->balance;
        }
        return null;
    }
    
    // 入金
    public function deposit(int $amount): bool {
        if ($amount > 0) {
            $this->balance += $amount;
            return true;
        }
        return false;
    }
    
    // 出金（privateメソッドを使用）
    public function withdraw(int $amount, string $pin): bool {
        if (!$this->verifyPin($pin)) {
            echo "PINコードが間違っています\n";
            return false;
        }
        
        if ($amount > $this->balance) {
            echo "残高不足です\n";
            return false;
        }
        
        $this->balance -= $amount;
        return true;
    }
    
    // private: クラス内からのみ呼び出せる
    private function verifyPin(string $pin): bool {
        return $this->pinCode === $pin;
    }
}

$account = new BankAccount("田中太郎", "123-456", "0000", 100000);

// publicには直接アクセスできる
echo "名義人: {$account->ownerName}\n";

// protectedには外部からアクセスできない
// echo $account->accountNumber;  // エラー！

// privateには外部からアクセスできない
// echo $account->balance;  // エラー！
// echo $account->pinCode;  // エラー！

// publicメソッド経由でアクセス
echo "残高: \{$account->getBalance()}\n";
echo "PIN認証残高: \{$account->getBalanceWithPin("0000")}\n";
echo "間違ったPIN: " . var_export($account->getBalanceWithPin("9999"), true) . "\n";

$account->deposit(50000);
echo "入金後の残高: \{$account->getBalance()}\n";

$account->withdraw(30000, "0000");
echo "出金後の残高: \{$account->getBalance()}\n";

// privateメソッドは外部から呼べない
// $account->verifyPin("0000");  // エラー！

?>

---
## 2. ゲッターとセッター

プロパティへのアクセスを制御するためのメソッドです。

### 設計思想：なぜゲッター/セッターを使うのか

1. **検証**: 不正な値を防げる
2. **計算値**: 値を計算して返せる
3. **イベント通知**: 値が変更されたときに処理を実行できる
4. **柔軟性**: 後から実装を変えても外側に影響しない

In [ ]:
<?php

class User {
    private string $name;
    private string $email;
    private int $age;
    private ?DateTime $updatedAt = null;
    
    public function __construct(string $name, string $email, int $age) {
        $this->name = $name;
        $this->setEmail($email);  // セッター経由でバリデーション
        $this->setAge($age);
    }
    
    // ゲッター：値を取得
    public function getName(): string {
        return $this->name;
    }
    
    public function getEmail(): string {
        return $this->email;
    }
    
    public function getAge(): int {
        return $this->age;
    }
    
    // セッター：値を設定（バリデーション付き）
    public function setName(string $name): void {
        if (empty(trim($name))) {
            throw new InvalidArgumentException("名前は空にできません");
        }
        if (mb_strlen($name) > 50) {
            throw new InvalidArgumentException("名前は50文字以下である必要があります");
        }
        $this->name = $name;
        $this->markAsUpdated();
    }
    
    public function setEmail(string $email): void {
        if (!filter_var($email, FILTER_VALIDATE_EMAIL)) {
            throw new InvalidArgumentException("無効なメールアドレス形式");
        }
        $this->email = $email;
        $this->markAsUpdated();
    }
    
    public function setAge(int $age): void {
        if ($age < 0 || $age > 150) {
            throw new InvalidArgumentException("年齢は0から150の間である必要があります");
        }
        $this->age = $age;
        $this->markAsUpdated();
    }
    
    // 計算値を返すゲッター
    public function getBirthYear(): int {
        return (int)date('Y') - $this->age;
    }
    
    public function isAdult(): bool {
        return $this->age >= 18;
    }
    
    public function getUpdatedAt(): ?DateTime {
        return $this->updatedAt;
    }
    
    // 値変更時のイベント
    private function markAsUpdated(): void {
        $this->updatedAt = new DateTime();
    }
}

// 使用例
$user = new User("山田太郎", "yamada@example.com", 30);

echo "名前: {$user->getName()}\n";
echo "メール: {$user->getEmail()}\n";
echo "年齢: {$user->getAge()}歳\n";
echo "生まれ年: 約{$user->getBirthYear()}年\n";
echo "成人: " . ($user->isAdult() ? 'はい' : 'いいえ') . "\n";

// 値の変更（セッター経由）
$user->setAge(31);
echo "変更後の年齢: {$user->getAge()}歳\n";
echo "更新時刻: " . $user->getUpdatedAt()->format('Y-m-d H:i:s') . "\n";

// バリデーションエラーの例
try {
    $user->setAge(-5);  // エラー！
} catch (InvalidArgumentException $e) {
    echo "エラー: {$e->getMessage()}\n";
}

try {
    $user->setEmail("invalid-email");  // エラー！
} catch (InvalidArgumentException $e) {
    echo "エラー: {$e->getMessage()}\n";
}

?>

---
## 3. PHP 8.4以降: プロパティフック

PHP 8.4では、ゲッター/セッターをより簡潔に書ける「プロパティフック」が導入されました。

In [ ]:
<?php

// PHP 8.4以降のプロパティフック（将来のために紹介）
// 現在のバージョンでは従来のゲッター/セッターを使用

class Temperature {
    private float $celsius;
    
    public function __construct(float $celsius) {
        $this->setCelsius($celsius);
    }
    
    public function getCelsius(): float {
        return $this->celsius;
    }
    
    public function setCelsius(float $value): void {
        if ($value < -273.15) {
            throw new InvalidArgumentException("絶対零度以下にはできません");
        }
        $this->celsius = $value;
    }
    
    // 計算値：華氏
    public function getFahrenheit(): float {
        return ($this->celsius * 9/5) + 32;
    }
    
    public function setFahrenheit(float $value): void {
        $this->setCelsius(($value - 32) * 5/9);
    }
    
    // 計算値：ケルビン
    public function getKelvin(): float {
        return $this->celsius + 273.15;
    }
}

$temp = new Temperature(25);
echo "摂氏: {$temp->getCelsius()}°C\n";
echo "華氏: {$temp->getFahrenheit()}°F\n";
echo "ケルビン: {$temp->getKelvin()}K\n";

$temp->setFahrenheit(68);
echo "\n68°Fを摂氏に変換: {$temp->getCelsius()}°C\n";

?>

---
## 4. 静的プロパティ・静的メソッド（static）

`static`キーワードを使うと、インスタンス化せずに直接クラスからアクセスできます。

**いつ使うか？**
- インスタンスに依存しない機能
- 共有のデータや設定
- ユーティリティ関数
- ファクトリーメソッド

In [ ]:
<?php

class Counter {
    // 静的プロパティ：すべてのインスタンスで共有される
    private static int $globalCount = 0;
    
    // インスタンスプロパティ：各インスタンス固有
    private int $instanceCount = 0;
    
    public function __construct() {
        self::$globalCount++;  // 静的プロパティをインクリメント
        $this->instanceCount++;
    }
    
    // 静的メソッド：インスタンス化せずに呼び出せる
    public static function getGlobalCount(): int {
        return self::$globalCount;
    }
    
    public function getInstanceCount(): int {
        return $this->instanceCount;
    }
    
    // 静的メソッドからは非静的メソッドを呼べない
    // public static function invalidMethod() {
    //     $this->instanceCount++;  // エラー！$thisは使えない
    // }
}

// インスタンス化せずに静的メソッドを呼び出す
echo "作成前のカウント: " . Counter::getGlobalCount() . "\n";

$counter1 = new Counter();
$counter2 = new Counter();
$counter3 = new Counter();

echo "作成後のグローバルカウント: " . Counter::getGlobalCount() . "\n";
echo "counter1のインスタンスカウント: {$counter1->getInstanceCount()}\n";
echo "counter2のインスタンスカウント: {$counter2->getInstanceCount()}\n";

?>

In [ ]:
<?php

/**
 * 静的メソッドの実践例：ユーティリティクラス
 */
class StringHelper {
    // 全て静的メソッド - インスタンス化不要
    
    public static function slugify(string $text): string {
        $text = strtolower($text);
        $text = preg_replace('/[^a-z0-9]+/', '-', $text);
        return trim($text, '-');
    }
    
    public static function truncate(string $text, int $length, string $suffix = '...'): string {
        if (mb_strlen($text) <= $length) {
            return $text;
        }
        return mb_substr($text, 0, $length) . $suffix;
    }
    
    public static function generateRandomString(int $length = 10): string {
        $characters = '0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ';
        $result = '';
        for ($i = 0; $i < $length; $i++) {
            $result .= $characters[random_int(0, strlen($characters) - 1)];
        }
        return $result;
    }
}

// 使い方：newなしで直接呼び出す
echo StringHelper::slugify("Hello World! This is a Test") . "\n";
echo StringHelper::truncate("これはとても長いテキストです。途中で切り取られます。", 15) . "\n";
echo StringHelper::generateRandomString(16) . "\n";

?>

---
## 5. マジックメソッド

PHPには`__`で始まる特別なメソッドがあります。
これらを「マジックメソッド」と呼びます。

| メソッド | 呼ばれるタイミング |
|---------|------------------|
| `__construct()` | インスタンス化時 |
| `__destruct()` | オブジェクト破棄時 |
| `__get()` | 存在しないプロパティ読み取り時 |
| `__set()` | 存在しないプロパティ設定時 |
| `__isset()` | isset() empty()呼び出し時 |
| `__unset()` | unset()呼び出し時 |
| `__call()` | 存在しないメソッド呼び出し時 |
| `__callStatic()` | 存在しない静的メソッド呼び出し時 |
| `__toString()` | オブジェクトを文字列に変換時 |

In [ ]:
<?php

class DynamicObject {
    private array $data = [];
    
    // 存在しないプロパティを読み取ろうとした時
    public function __get(string $name) {
        echo "プロパティ '{$name}' を取得\n";
        return $this->data[$name] ?? null;
    }
    
    // 存在しないプロパティを設定しようとした時
    public function __set(string $name, $value): void {
        echo "プロパティ '{$name}' に '{$value}' を設定\n";
        $this->data[$name] = $value;
    }
    
    // isset() が呼ばれた時
    public function __isset(string $name): bool {
        return isset($this->data[$name]);
    }
    
    // unset() が呼ばれた時
    public function __unset(string $name): void {
        unset($this->data[$name]);
        echo "プロパティ '{$name}' を削除\n";
    }
    
    // 存在しないメソッドが呼ばれた時
    public function __call(string $name, array $arguments) {
        echo "メソッド '{$name}' が引数 " . implode(', ', $arguments) . " で呼ばれました\n";
        
        // ゲッター風の呼び出しに対応
        if (str_starts_with($name, 'get')) {
            $property = lcfirst(substr($name, 3));
            return $this->data[$property] ?? null;
        }
        
        return null;
    }
    
    // オブジェクトを文字列に変換
    public function __toString(): string {
        $properties = [];
        foreach ($this->data as $key => $value) {
            $properties[] = "{$key}: {$value}";
        }
        return "DynamicObject(" . implode(', ', $properties) . ")";
    }
}

$obj = new DynamicObject();

// 動的なプロパティ設定
$obj->name = "田中太郎";
$obj->age = 30;
$obj->city = "東京";

// 動的なプロパティ取得
echo "名前: {$obj->name}\n";
echo "年齢: {$obj->age}\n";

// isset のテスト
echo "nameは存在" . (isset($obj->name) ? "します" : "しません") . "\n";
echo "emailは存在" . (isset($obj->email) ? "します" : "しません") . "\n";

// unset のテスト
unset($obj->city);

// 動的なメソッド呼び出し
$obj->someMethod("Hello", "World");
$obj->getName();  // ゲッター風

// 文字列への変換
echo "\nオブジェクト: " . $obj . "\n";

?>

---
## 6. const と static の違い

| | const | static |
|---|-------|--------|
| 値の変更 | ×（定数） | ○ |
| 型宣言 | ○ | ○ |
| アクセス範囲 | publicのみ | public/protected/private |
|

**使い分け**
- `const`: 変更不要な設定値（エラーコード、デフォルト値など）
- `static`: 変更可能な共有データ（カウンター、設定など）

In [ ]:
<?php

class AppConfig {
    // 定数：変更不要な値
    public const APP_NAME = 'My Application';
    public const APP_VERSION = '1.0.0';
    public const MAX_LOGIN_ATTEMPTS = 5;
    public const SESSION_TIMEOUT = 3600;  // 秒
    
    // 静的プロパティ：変更可能な値
    private static string $environment = 'production';
    private static bool $debugMode = false;
    private static array $config = [];
    
    // 静的メソッド
    public static function setEnvironment(string $env): void {
        self::$environment = $env;
    }
    
    public static function getEnvironment(): string {
        return self::$environment;
    }
    
    public static function setDebugMode(bool $enabled): void {
        self::$debugMode = $enabled;
    }
    
    public static function isDebugMode(): bool {
        return self::$debugMode;
    }
    
    public static function set(string $key, $value): void {
        self::$config[$key] = $value;
    }
    
    public static function get(string $key, $default = null) {
        return self::$config[$key] ?? $default;
    }
    
    // 定数を使ったメソッド
    public static function getSessionTimeout(): int {
        return self::SESSION_TIMEOUT;
    }
}

// 定数の使用
echo "アプリ名: " . AppConfig::APP_NAME . "\n";
echo "バージョン: " . AppConfig::APP_VERSION . "\n";

// 静的プロパティの使用
AppConfig::setEnvironment('development');
AppConfig::setDebugMode(true);
echo "環境: " . AppConfig::getEnvironment() . "\n";
echo "デバッグモード: " . (AppConfig::isDebugMode() ? 'ON' : 'OFF') . "\n";

// 動的な設定
AppConfig::set('db.host', 'localhost');
AppConfig::set('db.port', 3306);
echo "DBホスト: " . AppConfig::get('db.host') . "\n";
echo "セッションタイムアウト: " . AppConfig::getSessionTimeout() . "秒\n";

?>

---
## 7. 実践例：リポジトリパターン

プロパティとメソッドを使った実践的な設計例です。

In [ ]:
<?php

/**
 * エンティティ：データを表すクラス
 */
class TaskEntity {
    private ?int $id = null;
    private string $title;
    private string $status;
    private int $priority;
    private readonly DateTime $createdAt;
    private ?DateTime $completedAt = null;
    
    public function __construct(string $title, int $priority = 1) {
        $this->title = $title;
        $this->status = 'pending';
        $this->priority = $priority;
        $this->createdAt = new DateTime();
    }
    
    // ゲッター
    public function getId(): ?int {
        return $this->id;
    }
    
    public function getTitle(): string {
        return $this->title;
    }
    
    public function getStatus(): string {
        return $this->status;
    }
    
    public function getPriority(): int {
        return $this->priority;
    }
    
    public function getCreatedAt(): DateTime {
        return clone $this->createdAt;  // 不変性を保つためクローンを返す
    }
    
    // セッター（プライベート：リポジトリからのみ変更可能）
    public function setId(int $id): void {
        if ($this->id !== null) {
            throw new RuntimeException("IDは一度設定すると変更できません");
        }
        $this->id = $id;
    }
    
    public function setStatus(string $status): void {
        $allowedStatuses = ['pending', 'in_progress', 'completed', 'cancelled'];
        if (!in_array($status, $allowedStatuses)) {
            throw new InvalidArgumentException("無効なステータス: {$status}");
        }
        $this->status = $status;
        if ($status === 'completed') {
            $this->completedAt = new DateTime();
        }
    }
    
    // 読み取り専用プロパティ相当のメソッド
    public function isCompleted(): bool {
        return $this->status === 'completed';
    }
    
    // 文字列表現
    public function __toString(): string {
        $idStr = $this->id !== null ? "#{$this->id}" : "(新規)";
        return "{$idStr} {$this->title} [{$this->status}] 優先度:{$this->priority}";
    }
}

/**
 * リポジトリ：データアクセスを担当するクラス
 */
class TaskRepository {
    // 依存関係を注入（DIコンテナから受け取る想定）
    private array $tasks = [];
    private int $nextId = 1;
    
    // タスクを保存
    public function save(TaskEntity $task): TaskEntity {
        if ($task->getId() === null) {
            // 新規タスク：IDを割り振る
            $task->setId($this->nextId++);
        }
        $this->tasks[$task->getId()] = $task;
        return $task;
    }
    
    // IDで検索
    public function findById(int $id): ?TaskEntity {
        return $this->tasks[$id] ?? null;
    }
    
    // 全件取得
    public function findAll(): array {
        return array_values($this->tasks);
    }
    
    // ステータスでフィルタ
    public function findByStatus(string $status): array {
        return array_filter($this->tasks, function($task) use ($status) {
            return $task->getStatus() === $status;
        });
    }
    
    // 削除
    public function delete(TaskEntity $task): bool {
        $id = $task->getId();
        if ($id === null) {
            return false;
        }
        unset($this->tasks[$id]);
        return true;
    }
}

// 使用例
$repository = new TaskRepository();

// タスク作成
$task1 = new TaskEntity("メールを返信する", 3);
$task2 = new TaskEntity("レポートを書く", 1);
$task3 = new TaskEntity("会議の準備", 2);

// 保存（自動でIDが割り振られる）
$repository->save($task1);
$repository->save($task2);
$repository->save($task3);

echo $task1 . "\n";  // #1 メールを返信する [pending] 優先度:3

// ステータス変更
$task1->setStatus('in_progress');
$task1->setStatus('completed');
echo $task1 . "\n";

// 検索
$found = $repository->findById(2);
echo "見つかったタスク: {$found}\n";

?>

---
## 📝 実践問題

### 問題1: 不変クラス

一度作成したら変更できない不変クラスを作成してください。

- `Money` クラス（金額と通貨）
- `add()` `subtract()` `multiply()` メソッドは新しいインスタンスを返す
- 金額が負にならないようにする

### 問題2: バリューオブジェクト

以下のバリューオブジェクトを作成してください。

- `Email` クラス（バリデーション付き）
- `PhoneNumber` クラス（フォーマット検証）
- `DateRange` クラス（開始日と終了日、期間チェック）

### 問題3: ビルダーパターン

複雑なオブジェクトを構築するビルダークラスを作成してください。

- `QueryBuilder` クラス
- `select()` `where()` `orderBy()` `limit()` メソッド
- `build()` でSQL文字列を生成
- メソッドチェーン対応

---
## まとめ

### ✅ ポイントチェック

- [ ] public/protected/privateの違いを理解した
- [ ] ゲッター/セッターを使えるようになった
- [ ] staticの使い方をマスターした
- [ ] マジックメソッドの基礎を学んだ
- [ ] constとstaticの違いを理解した
- [ ] カプセル化の重要性を理解した

### 🎯 プロパティ・メソッド設計の原則

1. **最小限の公開** - 必要以上にpublicにしない
2. **不変性を重視** - readonlyやfinalを活用する
3. "**Tell, Don't Ask**" - オブジェクトに尋ねるではなく、命じる
4. **自己完結** - オブジェクトは自分のデータを自分で管理する

### 🚀 次のステップ

次は第14章で、継承とポリモーフィズムについて学びましょう！

---

**関連リソース**
- [PHP: アクセス権](https://www.php.net/manual/ja/language.oop5.visibility.php)
- [PHP: マジックメソッド](https://www.php.net/manual/ja/language.oop5.magic.php)
- [PHP: staticキーワード](https://www.php.net/manual/ja/language.oop5.static.php)